In [1]:
# Preamble
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from process_baywheels_tripdata import *
import geopandas as gpd
#import geoplot
from geopy import distance
from shapely import wkt
from shapely.geometry.point import Point
import seaborn as sns
import folium

In [2]:
csv_filepath = '201909-baywheels-tripdata.csv'
# df = pd.read_csv(csv_filepath)

df = get_df_from_csv(csv_filepath)
stations = get_unique_stations(df)
# Create a grid that shows O-D analysis: rows represent origin, columns represent destination

stations['location'] = stations[['long', 'lat']].apply(lambda x: Point(x[0], x[1]), axis=1)
stations = stations.drop(['long','lat'], axis=1)
gs = gpd.GeoDataFrame(stations, geometry='location')
gs.crs = {'init' : 'epsg:4326'}

In [3]:
# Limit to stations within SF
sf_lat = (37.73, 37.81)
sf_long = (-122.47, -122.359720)

df.iloc[0]
within_sf = (sf_lat[0] <= df.start_station_latitude) & (df.start_station_latitude <= sf_lat[1]) \
            & (sf_long[0] <= df.start_station_longitude) & (df.start_station_longitude <= sf_long[1]) \
            & (sf_lat[0] <= df.end_station_latitude) & (df.end_station_latitude <= sf_lat[1]) \
            & (sf_long[0] <= df.end_station_longitude) & (df.end_station_longitude <= sf_long[1])

df = df[within_sf]

In [6]:
gp = df.groupby(['start_station_name', 'end_station_name'])
od_counts = gp.count()['duration_sec']
od = od_counts.unstack()

top_routes = od_counts.sort_values(ascending=False).head(50)
top_stations = top_routes.index.to_frame()
top_stations = pd.concat([top_stations.start_station_name, top_stations.end_station_name]).unique()
station_list = sorted(list(top_stations))

## Map of Bikeshare Destinations by Station of Origin

In [7]:
# Bikeshare Origin-Destination Map

# This has to be opened in a separate browser tab when you allow for all stations
m = folium.Map(location=[37.77, -122.4], 
                tiles='CartoDB positron', # Stamen watercolor is gorgeous
               zoom_start=14)

def plot_station_layer(m, od, station):
    od_df = od.loc[station].to_frame()\
    .merge(stations, left_on='end_station_name', right_on='name')

    od_df['markersize'] = od_df[station] * 5

    destination = od_df.to_dict(orient='records')

    def point_to_long_lat(point):
        return (point.y, point.x)
    
    # Make a feature group for each origin
    layer1 = folium.map.FeatureGroup(name=station, overlay=False)
    layer1.add_child(folium.raster_layers.TileLayer(location=[37.77, -122.4], 
                tiles='CartoDB positron', zoom_start=13))
    # Plot the origin
    folium.Marker(location=point_to_long_lat(
                                stations[stations.name == station].location.iloc[0]),
                    icon=folium.map.Icon(icon='times',
                                prefix='fa', color='red'),
                    tooltip=station
                 ).add_to(layer1)

    # Plot the destinations
    for d in destination:
        col = '#377eb8'
        if d['name'] == station:
            col = '#e41a1c'
        loc = (d['location'].y, d['location'].x)
        folium.CircleMarker(location=loc,
                            color=col,
                           radius=d[station]/9,
                            fill=True,
                            opacity=0.8,
                            fill_opacity=0.8,
                            tooltip=d['name'] + ': ' + str(d[station])
                           ).add_to(layer1)
    m.add_child(layer1)

for station in station_list:
    try:
        plot_station_layer(m, od, station)
    except KeyError as e:
        print('Found a KeyError on this station: ' + station)
        continue

# Add layer control
folium.map.LayerControl(collapsed=False).add_to(m)

m.save('bikeshare_station_origin.html')

## Map of Bikeshare Arrivals and Departures by Hour

In [8]:
df['start_hr'] = df.start_time_hr.apply(lambda x: int(x.strftime('%H')))
df['end_hr'] = df.end_time_hr.apply(lambda x: int(x.strftime('%H')))

sg = df.groupby(['start_station_id', 'start_hr', 'start_time_hr'])
eg = df.groupby(['end_station_id', 'end_hr', 'end_time_hr'])

# For departures and arrivals, get the average per station per hour of day
depart = sg['start_station_id'].count()
depart = depart.to_frame()\
    .rename({'start_station_id': 'count'}, axis=1).fillna(0)\
    .reset_index()
depart = depart.groupby(['start_station_id', 'start_hr'])['count']\
    .mean().to_frame().reset_index()\
    .rename({'start_station_id': 'station_id'}, axis=1)
depart = depart.reset_index().merge(stations, left_on='station_id', right_on='id')

arrive = eg['end_station_id'].count()
arrive = arrive.to_frame()\
    .rename({'end_station_id': 'count'}, axis=1).fillna(0)\
    .reset_index()
arrive = arrive.groupby(['end_station_id', 'end_hr'])['count']\
    .mean().to_frame().reset_index()\
    .rename({'end_station_id': 'station_id'}, axis=1)
arrive = arrive.reset_index().merge(stations, left_on='station_id', right_on='id')

In [9]:
m = folium.Map(location=[37.77, -122.4], 
                tiles='CartoDB positron',
               name='_',
               zoom_start=14)


def point_to_long_lat_depart(point):
    return (point.y, point.x - 0.0001)

def point_to_long_lat_arrive(point):
    return (point.y, point.x + 0.0001)

def plot_hour(hr):
    depart2 = depart[depart['start_hr'] == hr].to_dict(ood = od_counts.unstack()rient='records')
    arrive2 = arrive[arrive['end_hr'] == hr].to_dict(orient='records')
    # Make a feature group for each origin
    layer1 = folium.map.FeatureGroup(name='Hour {}'.format(str(hr).zfill(2)), overlay=False)
    shrink_factor = 1
    max_size = 35
    min_count = 2

    for d in depart2:
        val = round(d['count'], 2) 
        if val < min_count:
            continue
        col = '#B22222'
        loc = point_to_long_lat_depart(d['location'])
        folium.CircleMarker(location=loc,
                            color=col,
                            radius= max(0.1, min(max_size, val/shrink_factor)),
                            fill=True,
                            opacity=0.8,
                            fill_opacity=0.8,
                            tooltip=d['name'] + ': -' + str(val)
                            ).add_to(layer1)

    for d in arrive2:
        val = round(d['count'], 2)
        if val < min_count:
            continue
        col = '#4682B4'
        loc = point_to_long_lat_arrive(d['location'])
        folium.CircleMarker(location=loc,
                            color=col,
                            radius= max(0.1, min(max_size, val/shrink_factor)),
                            fill=True,
                            opacity=0.8,
                            fill_opacity=0.8,
                            tooltip=d['name'] + ': +' + str(val)
                            ).add_to(layer1)
    m.add_child(layer1)
    #m.add_child(layer2)
    
for hr in range(0,24,1):
    plot_hour(hr)
    
layer = folium.map.FeatureGroup(name='Map Overlay', overlay=True)
layer.add_child(folium.raster_layers.TileLayer(location=[37.77, -122.4], 
                tiles='CartoDB positron', zoom_start=13))
m.add_child(layer)
# Add layer control
folium.map.LayerControl(collapsed=False).add_to(m)
m.save('bikeshare_by_hour.html')